In [1]:
import csv
import pandas as pd

from ast import literal_eval

In [2]:
job_data = []
with open ('../bert-model/data/jobs_data.csv') as j_file:
    csv_reader = csv.reader(j_file)
    next(csv_reader)
    for row in csv_reader:
        row[2] = [skill.strip(" '\"") for skill in row[2].strip('[]').split(',')]
        job_data.append(row)

jobs_df = pd.DataFrame(job_data, columns=['title', 'description', 'skills'])

course_data = []
with open ('../bert-model/data/courses_data.csv') as c_data:
    csv_reader = csv.reader(c_data)
    next(csv_reader)
    for row in csv_reader:
        row[1] = [skill.strip(" '\"") for skill in row[1].strip('[]').split(',')]
        course_data.append(row)

courses_df = pd.DataFrame(course_data, columns=['course', 'skills'])

In [3]:
numskills_j_data = []

for i in range(0, len(job_data)):
    numskills_j_data.append(len(job_data[i][2]))
    

In [4]:
def get_all_acquired_skills(courses_df):
    all_acquired_skills = set() 
    
    for skills in courses_df['skills']:
        all_acquired_skills.update(skills)

    return all_acquired_skills

In [5]:
numskills_j_df = []
for _, row in jobs_df.iterrows():
    numskills_j_df.append(len(row['skills']))

In [6]:
def all_class_comparison(jobs_df, courses_df, all_course_skills):
    gaps_df = pd.DataFrame(columns=['job title', 'covered skills', 'missing skills', 'intersecting course skills', 'gt label']) #'nonrequired skills taught', 'gt label'])

    for _, job in jobs_df.iterrows():
        job_title = job['title']
        job_skills = set(job['skills'])
        courses = zip(courses_df['course'], courses_df['skills'])

        missing_skills = job_skills - all_course_skills
        covered_skills = job_skills.intersection(all_course_skills)
        num_missing = float(len(missing_skills))
        num_job_skills = float(len(job_skills))

        #non_required = []
        intersecting = []

        for course, skills in courses:
            course_inter = list(job_skills.intersection(set(skills)))
            if len(course_inter) > 0:
                intersecting.append((course, course_inter))
            #non_required.append((course, list(set(skills) - job_skills)))

        gaps_entry = pd.DataFrame([{
                                    'job title': job_title,
                                    'covered skills': sorted(covered_skills),
                                    'missing skills': sorted(missing_skills), 
                                    'intersecting course skills': sorted(intersecting), 
                                    #'nonrequired skills taught': non_required,
                                    'gt label': (1.0 - (num_missing/num_job_skills))      
                                }])
        
        gaps_df = pd.concat([gaps_df, gaps_entry], ignore_index=True)

        
    return gaps_df

In [7]:
all_course_skills = get_all_acquired_skills(courses_df)

gaps_df = all_class_comparison(jobs_df, courses_df, all_course_skills)

gaps_df

C:\Users\ayoun\AppData\Local\Temp\ipykernel_3772\1459243222.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gaps_df = pd.concat([gaps_df, gaps_entry], ignore_index=True)


,job title,covered skills,missing skills,intersecting course skills,gt label
0,Adobe_AI_ML_Engineer,"[Communication, Debugging, Python, Quality Ass...","[Analytical skills, Angular, Bug fixing, Code ...","[(CS150, [Python]), (CS164, [Debugging, Qualit...",0.193548
1,Adobe_Junior_SDE,"[Java, Security, Teamwork]","[Adobe Commerce, Agile development, Architectu...","[(CS110, [Security]), (CS150, [Security]), (CS...",0.071429
2,Adobe_Software_Engineering_Intern,"[Algorithms, C++, Communication, Java, Network...","[Adobe Experience Manager, Analytical skills, ...","[(CS150, [Python]), (CS152, [Algorithms]), (CS...",0.333333
3,Adobe_Software_Quality_Engineer,"[Communication, Quality Assurance, Testing]","[Analytical skills, Collaboration, Deployment,...","[(CS164, [Quality Assurance, Testing]), (CS314...",0.150000
4,Adobe_Software_Solutions_Architect,"[Communication, Optimization, Research, Securi...","[Actionable Insights, Adobe Cloud Solutions, A...","[(CS110, [Security, Research]), (CS150, [Secur...",0.094340
...,...,...,...,...,...
74,Tesla_Software_Compliance_Engineer,"[C, C++, Communication, Debugging, Java, Pytho...","[C#, Collaboration, Compute technologies, Data...","[(CS150, [Python]), (CS162, [Java]), (CS164, [...",0.184211
75,Tesla_Software_Engineer_Firmware,"[C, Communication, Debugging, Software Design,...","[Algorithm Development, Analytical skills, Ass...","[(CS164, [Debugging]), (CS214, [Debugging, Sof...",0.151515
76,Tesla_Software_Engineer_Recruiting,"[Design Patterns, Optimization, Web Applications]","[Code quality, Collaboration, Compute technolo...","[(CS312, [Web Applications]), (CS314, [Optimiz...",0.157895
77,Yahoo_Frontend_Software_Dev_Engineer,"[Java, Python, Regression Testing, Teamwork, W...","[Ad hoc analyses, Analytical skills, Business ...","[(CS150, [Python]), (CS162, [Java]), (CS164, [...",0.131579


In [8]:
numskills_gaps_df = []
for _, row in gaps_df.iterrows():
    numskills_gaps_df.append(len(row['missing skills']) + len(row['covered skills']))

print(numskills_j_data)
print(numskills_j_df)
print(numskills_gaps_df)
print(numskills_j_data == numskills_j_df)
print(numskills_j_data == numskills_gaps_df)

[31, 42, 21, 20, 53, 38, 28, 28, 26, 29, 27, 34, 34, 49, 54, 10, 37, 24, 32, 20, 39, 47, 48, 31, 25, 40, 20, 17, 11, 27, 10, 6, 12, 8, 4, 10, 7, 9, 7, 10, 11, 16, 14, 12, 7, 12, 24, 34, 38, 19, 27, 22, 23, 31, 34, 24, 52, 46, 53, 27, 24, 41, 23, 20, 45, 26, 30, 43, 16, 43, 24, 29, 39, 35, 38, 33, 19, 38, 38]
[31, 42, 21, 20, 53, 38, 28, 28, 26, 29, 27, 34, 34, 49, 54, 10, 37, 24, 32, 20, 39, 47, 48, 31, 25, 40, 20, 17, 11, 27, 10, 6, 12, 8, 4, 10, 7, 9, 7, 10, 11, 16, 14, 12, 7, 12, 24, 34, 38, 19, 27, 22, 23, 31, 34, 24, 52, 46, 53, 27, 24, 41, 23, 20, 45, 26, 30, 43, 16, 43, 24, 29, 39, 35, 38, 33, 19, 38, 38]
[31, 42, 21, 20, 53, 38, 28, 28, 26, 29, 27, 34, 34, 49, 54, 10, 37, 24, 32, 20, 39, 47, 48, 31, 25, 40, 20, 17, 11, 27, 10, 6, 12, 8, 4, 10, 7, 9, 7, 10, 11, 16, 14, 12, 7, 12, 24, 34, 38, 19, 27, 22, 23, 31, 34, 24, 52, 46, 53, 27, 24, 41, 23, 20, 45, 26, 30, 43, 16, 43, 24, 29, 39, 35, 38, 33, 19, 38, 38]
True
True


In [9]:
gaps_df = gaps_df.sort_values(by='job title').reset_index(drop=True)
gaps_df.to_csv('skill_comparison.csv', index=False)

In [10]:
gaps_df.to_json('skill_comparison.json', orient='records', indent=2)

In [11]:
from collections import defaultdict

def skills_by_course(jobs_df, courses_df, all_course_skills):
    skills_taught = defaultdict(
        lambda: defaultdict(
            lambda: defaultdict(list)
        )
    )
    course_entries = list(zip(courses_df["course"], courses_df["skills"]))

    for _, job in jobs_df.iterrows():
        job_title = job['title']
        job_skills = set(job['skills'])
        intersecting_skills = job_skills.intersection(all_course_skills)

        if len(intersecting_skills) == 0:
            skills_taught[job_title]['status'] = 'None of the required skills are taught in any course'
        else:
            for rs in job_skills:

                courses_taught = sorted([course for course, skills in course_entries if rs in skills])

                num_taught = len(courses_taught)
                if num_taught == 0:
                    skills_taught[job_title]['required_skills'][rs] = 'None'
                elif num_taught == 1:
                    skills_taught[job_title]['required_skills'][rs] = courses_taught[0]
                else:
                    skills_taught[job_title]['required_skills'][rs] = courses_taught
        
    return skills_taught



In [12]:
skills_taught = skills_by_course(jobs_df, courses_df, all_course_skills)

In [13]:
import json

def default_to_regular_dict(d):
    if isinstance(d, defaultdict):
        d = {k: default_to_regular_dict(v) for k, v in d.items()}
    return d

skills_taught_regular = default_to_regular_dict(skills_taught)
with open("skills_by_course.json", "w") as f:
    json.dump(skills_taught_regular, f, indent=4)